# Evaluation of Multi-Agent approach using an orchestrator

In [1]:
import sys
import os
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates")
from dotenv import load_dotenv

sys.path.insert(0, os.path.abspath("../src"))
print(sys.path)

load_dotenv()
from approach_evaluator import ApproachEvaluator
from orchestrator_agent import create_sql_from_natural_text, visualize_graph
visualize_graph()

['/Users/yannikhahn/Code/LLMsAgents-TextToSQL/src', '/Users/yannikhahn/Code/LLMsAgents-TextToSQL/evaluation/src', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/yannikhahn/Library/Caches/pypoetry/virtualenvs/llmsagents-texttosql-JgGAmzxu-py3.12/lib/python3.12/site-packages', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates']


ModuleNotFoundError: No module named 'orchestrator_agent'

In [2]:
import pandas as pd

data_dir = os.getenv("DATA_DIRECTORY")
sql_input_file = pd.read_csv("../sample/dev.csv").sample(3)

# Config is just for logging purposes and does not affect the run
run_config = {
    "k_shot_prompting_creator_agent": 2,
    "k_shot_prompting_fixer_agent": 1,
    "k_shot_prompting_feedback_agent": 2,
    "k_shot_prompting_select_relevant_database": 0,
    "k_shot_prompting_select_relevant_tables": 2
}

evaluator = ApproachEvaluator(
    run_config=run_config,
    question_ids=sql_input_file["question_id"],
    approach=create_sql_from_natural_text
)
evaluator.evaluate()

{'query_result': '[(0,)]', 'is_correct': False, 'feedback': "The query did not return any results, indicating there might be an issue with the filtering conditions or field names. The 'OpenDate' field is not recognized, and the 'StatusType' field should be checked for accuracy.", 'updated_query': "SELECT COUNT(*) FROM california_schools WHERE OpenDate BETWEEN '2000-01-01' AND '2005-12-31' AND County = 'Stanislaus' AND StatusType = 'Directly Funded School';"}
{'query_result': '[(0,)]', 'is_correct': False, 'feedback': "The query did not return any results, indicating there might be an issue with the filtering conditions or field names. The 'OpenDate' field is not recognized, and the 'StatusType' field should be checked for accuracy.", 'updated_query': "SELECT COUNT(*) FROM california_schools WHERE OpenDate BETWEEN '2000-01-01' AND '2005-12-31' AND County = 'Stanislaus' AND StatusType = 'Directly Funded School';"}
Orchestrator Agent: An error occurred: 'feedback'
Error with final query: 

{'question_id': [489, 66, 442],
 'predicted_query': ['',
  'SELECT "baseSetSize", "code" FROM "sets" WHERE "block" = \'Masques\' OR "block" = \'Mirage\';',
  'SELECT "Column name: LinkTypeId" FROM postLinks WHERE "Column name: PostId" = \'PKHC\';'],
 'predicted_database': ['', 'card_games', 'codebase_community'],
 'feedbacks': [[],
  [{'query_result': '[]',
    'is_correct': False,
    'feedback': "The query did not return any results because the condition 'block = 'Masques' AND block = 'Mirage'' is contradictory. It's not possible for a single row to have both 'block' values. You should use 'OR' instead of 'AND' to retrieve sets from either 'Masques' or 'Mirage'.",
    'updated_query': "SELECT baseSetSize AS 'Set size', code AS 'Set code' FROM sets WHERE block IN ('Masques', 'Mirage');"},
   {'query_result': "[(350, 'MIR'), (350, 'MMQ'), (143, 'NEM'), (144, 'PCY'), (1, 'PMMQ'), (1, 'PNEM'), (1, 'PPCY'), (167, 'VIS'), (167, 'WTH')]",
    'is_correct': False,
    'feedback': "The query 

In [2]:
import pandas as pd
run = pd.read_csv("./results/results_full_run_291124/results.csv")

run.head()

,input_texts,predicted_query,predicted_database,target_queries,is_correct
0,Among the cards with converted mana cost highe...,SELECT COUNT(*) FROM cards WHERE convertedMana...,card_games,SELECT SUM(CASE WHEN T1.power LIKE '*' OR T1.p...,0
1,Calculate the average home team goal in the 20...,SELECT AVG(home_team_goal) FROM Match WHERE se...,european_football_2,SELECT CAST(SUM(t2.home_team_goal) AS REAL) / ...,1
2,Which college do most of the members go to?,SQL\r\n\r\nSELECT college\r\nFROM major\r\nWHE...,student_club,SELECT T2.college FROM member AS T1 INNER JOIN...,0
3,Calculate the percentage of molecules containi...,SELECT (CAST(COUNT(DISTINCT(molecule_id)) AS R...,toxicology,SELECT CAST(COUNT(CASE WHEN T1.element = 'h' A...,0
4,"Provide the related post title of ""How to tell...","SELECT ""Post Title"" FROM 'thrombosis_predictio...",thrombosis_prediction,SELECT T3.Title FROM postLinks AS T1 INNER JOI...,0


In [3]:
from approach_evaluator import valid_efficiency_score

ves = valid_efficiency_score(run)

print(f"Valid Efficiency Score: {ves}")

Valid Efficiency Score: 0.19128302335913877
